In [52]:
import json
from tqdm import tqdm
from prompts import system_prompt
import datasets
import os
from PIL import Image

In [48]:
str_template = """1. Basic Information
```json
{}
```

2. Critical Objects
```json
{}
```

3. Decision Analysis
{}

4. Meta Action
{}"""

In [50]:
results = []
json_file = "../../downloads/raw_annotations/positive/bus-only_lane/bus-only_lane_annotations_v5.json"
raw_data = json.load(open(json_file))
for current_data in tqdm(raw_data):
    part1 = {
        "scene_summary": current_data["scene_summary"],
        "weather": current_data["weather"],
        "time": current_data["time"],
        "road_environment": current_data["road_environment"],
        "ego_lane_position": current_data["ego_lane_position"],
    }
    part2 = [
        {
            "object_name": ann["static_attributes"],
            "2d_bbox": ann["2d_bbox"],
            "description": ann["influence"],
        }
        for ann in current_data["annotations"][0]["key_object_infos"]
    ]
    for x in part2:
        for i, y in enumerate(x["2d_bbox"]):
            if i % 2 == 0:
                x["2d_bbox"][i] = int(x["2d_bbox"][i] / 3840 * 1000)
            else:
                x["2d_bbox"][i] = int(x["2d_bbox"][i] / 2160 * 1000)
    part3 = current_data["decision_description"]
    part4 = current_data["meta_action"]
    response = str_template.format(part1, part2, part3, part4)
    image_path = os.path.join(
        "../../downloads/raw_annotations/positive/bus-only_lane/images",
        current_data["annotations"][0]["image_id"],
    )
    images = [Image.open(image_path).convert("RGB").resize((448 * 2, 448))]
    messages = [
        {"role": "system", "content": [{"text": system_prompt, "type": "text"}]},
        {"role": "user", "content": [{"text": None, "type": "image"}]},
        {"role": "assistant", "content": [{"text": response, "type": "text"}]},
    ]
    results.append({"images": images, "messages": messages})

100%|██████████| 57/57 [00:03<00:00, 17.79it/s]


In [53]:
d = datasets.Dataset.from_list(results)

In [55]:
d.save_to_disk("bus_only_sign")

Saving the dataset (0/1 shards):   0%|          | 0/57 [00:00<?, ? examples/s]